# Collecting Data for Final Project
Author: Tiffany Lin

Table of Contents:



Making sure duckdb is installed

In [1]:
import duckdb

# Placeholder for the database connection. It will be initialized later with the URL.
conn = duckdb.connect()
conn

Enabling HTTPFS

In [2]:
# Install and Load the HTTPFS extension
# This is required to access remote files over the web (HTTP/S)
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

Connect to the remote database source

In [3]:
# This is one of the several DuckDB databases hosted in the CS server.
# This database had the DPDP data for the year 2024 for the United States (en).
database_url = "https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb"

# Attach the remote file as a database named 'web_db' and start using it
try:
    conn.execute(f"ATTACH '{database_url}' AS web_db (READ_ONLY);")
    conn.execute("USE web_db;")
    print(f"Successfully attached database from: {database_url}")
except Exception as e:
    print(f"Error attaching database: {e}")

Successfully attached database from: https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb


Checking what is in the database

In [4]:
query = "PRAGMA show_tables"
result = conn.sql(query)
result

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ data_table │
└────────────┘

In [5]:
query = "SHOW DATABASES;"
result = conn.sql(query)
result

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ memory        │
│ web_db        │
└───────────────┘

In [ ]:
query2 = """
SELECT
    t1.date,
    t1.article,
    t1.qid,
    t1.pageviews AS daily_pageviews,
    t2.monthly_total
FROM data_table t1
JOIN (
    SELECT
        article,
        SUM(pageviews) AS monthly_total
    FROM data_table
    WHERE MONTH(date) = 4
      AND article LIKE '%song)'
    GROUP BY article
) t2 ON t1.article = t2.article
WHERE MONTH(t1.date) = 4
    AND t1.article LIKE '%song%'
ORDER BY t2.monthly_total DESC, t1.date;
"""
result_2 = conn.sql(query2).df()
result_2

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,date,article,qid,daily_pageviews,monthly_total
0,2024-04-18,Clara_Bow_(song),None,2603,471741.0
1,2024-04-19,Clara_Bow_(song),None,3599,471741.0
2,2024-04-19,Clara_Bow_(song),Q124426354,238580,471741.0
3,2024-04-20,Clara_Bow_(song),Q124426354,105861,471741.0
4,2024-04-21,Clara_Bow_(song),Q124426354,59858,471741.0
...,...,...,...,...,...
21047,2024-04-14,When_I'm_Gone_(Eminem_song),Q2263502,90,90.0
21048,2024-04-19,Bam_Bam_(Sister_Nancy_song),Q39069690,90,90.0
21049,2024-04-22,Poison_(Alice_Cooper_song),Q508899,90,90.0
21050,2024-04-26,Shelter_(Porter_Robinson_and_Madeon_song),Q27467421,90,90.0


In [5]:
result_2

,date,article,qid,daily_pageviews,monthly_total
0,2024-04-18,Clara_Bow_(song),None,2603,471741.0
1,2024-04-19,Clara_Bow_(song),None,3599,471741.0
2,2024-04-19,Clara_Bow_(song),Q124426354,238580,471741.0
3,2024-04-20,Clara_Bow_(song),Q124426354,105861,471741.0
4,2024-04-21,Clara_Bow_(song),Q124426354,59858,471741.0
...,...,...,...,...,...
21047,2024-04-14,When_I'm_Gone_(Eminem_song),Q2263502,90,90.0
21048,2024-04-19,Bam_Bam_(Sister_Nancy_song),Q39069690,90,90.0
21049,2024-04-22,Poison_(Alice_Cooper_song),Q508899,90,90.0
21050,2024-04-26,Shelter_(Porter_Robinson_and_Madeon_song),Q27467421,90,90.0


In [14]:
song_df = result_2.groupby("article")
print(song_df)

Trying to get a more simplified query

In [8]:
query3 = """
SELECT
    article,
    SUM(pageviews) AS monthly_total
FROM data_table
WHERE MONTH(date) = 4
    AND article LIKE '%song)'
GROUP BY article
ORDER BY monthly_total DESC
LIMIT 100;
"""
result_3 = conn.sql(query3).df()
result_3

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,article,monthly_total
0,Clara_Bow_(song),471741.0
1,Blackbird_(Beatles_song),64358.0
2,Fortnight_(Taylor_Swift_song),56037.0
3,Karma_(JoJo_Siwa_song),38489.0
4,Texas_Hold_'Em_(song),36496.0
...,...,...
95,Time_After_Time_(Cyndi_Lauper_song),7381.0
96,It's_My_Life_(Talk_Talk_song),7280.0
97,Redrum_(21_Savage_song),7277.0
98,Believe_(Cher_song),7247.0


In [8]:
query4 = """
SELECT DISTINCT
    article,
    qid
FROM data_table
WHERE qid IS NOT NULL
  AND article LIKE '%song)';
"""
result_4 = conn.sql(query4).df()
result_4

,article,qid
0,When_a_Man_Loves_a_Woman_(song),Q2456757
1,Rocket_Man_(song),Q56070897
2,Shia_LaBeouf_(song),Q25206229
3,Let_It_Be_(Beatles_song),Q184259
4,The_Man_Who_Sold_the_World_(song),Q2341319
...,...,...
4541,Busy_Woman_(Sabrina_Carpenter_song),Q126327826
4542,More_than_This_(One_Direction_song),Q932456
4543,That_Lady_(song),Q7711220
4544,November_18th_(song),Q125503906


In [9]:
import pandas as pd

In [10]:
qid_lookup = result_4[['article', 'qid']].drop_duplicates(subset=['article'])

In [11]:
qid_lookup

,article,qid
0,When_a_Man_Loves_a_Woman_(song),Q2456757
1,Rocket_Man_(song),Q56070897
2,Shia_LaBeouf_(song),Q25206229
3,Let_It_Be_(Beatles_song),Q184259
4,The_Man_Who_Sold_the_World_(song),Q2341319
...,...,...
4541,Busy_Woman_(Sabrina_Carpenter_song),Q126327826
4542,More_than_This_(One_Direction_song),Q932456
4543,That_Lady_(song),Q7711220
4544,November_18th_(song),Q125503906


In [12]:
df_merged = pd.merge(result_2, qid_lookup, 
                     on='article', 
                     how='left')

In [13]:
df_merged

,date,article,qid_x,daily_pageviews,monthly_total,qid_y
0,2024-04-18,Clara_Bow_(song),None,2603,471741.0,Q124426354
1,2024-04-19,Clara_Bow_(song),None,3599,471741.0,Q124426354
2,2024-04-19,Clara_Bow_(song),Q124426354,238580,471741.0,Q124426354
3,2024-04-20,Clara_Bow_(song),Q124426354,105861,471741.0,Q124426354
4,2024-04-21,Clara_Bow_(song),Q124426354,59858,471741.0,Q124426354
...,...,...,...,...,...,...
21047,2024-04-14,When_I'm_Gone_(Eminem_song),Q2263502,90,90.0,Q2263502
21048,2024-04-19,Bam_Bam_(Sister_Nancy_song),Q39069690,90,90.0,Q39069690
21049,2024-04-22,Poison_(Alice_Cooper_song),Q508899,90,90.0,Q508899
21050,2024-04-26,Shelter_(Porter_Robinson_and_Madeon_song),Q27467421,90,90.0,Q27467421


In [15]:
df_new = df_merged.drop(['date','qid_x', 'daily_pageviews'], axis=1)

In [16]:
df_new

,article,monthly_total,qid_y
0,Clara_Bow_(song),471741.0,Q124426354
1,Clara_Bow_(song),471741.0,Q124426354
2,Clara_Bow_(song),471741.0,Q124426354
3,Clara_Bow_(song),471741.0,Q124426354
4,Clara_Bow_(song),471741.0,Q124426354
...,...,...,...
21047,In_the_Garden_(1912_song),90.0,Q11977017
21048,Bam_Bam_(Sister_Nancy_song),90.0,Q39069690
21049,Poison_(Alice_Cooper_song),90.0,Q508899
21050,Shelter_(Porter_Robinson_and_Madeon_song),90.0,Q27467421


In [17]:
df_clean = df_new.drop_duplicates()

In [18]:
df_clean

,article,monthly_total,qid_y
0,Clara_Bow_(song),471741.0,Q124426354
15,Blackbird_(Beatles_song),64358.0,Q880719
45,Fortnight_(Taylor_Swift_song),56037.0,Q125511018
58,Karma_(JoJo_Siwa_song),38489.0,Q125450884
69,Texas_Hold_'Em_(song),36496.0,Q124507936
...,...,...,...
21047,In_the_Garden_(1912_song),90.0,Q11977017
21048,Bam_Bam_(Sister_Nancy_song),90.0,Q39069690
21049,Poison_(Alice_Cooper_song),90.0,Q508899
21050,Shelter_(Porter_Robinson_and_Madeon_song),90.0,Q27467421


In [19]:
df_clean_new = df_clean.rename(columns={'monthly_total': 'pageviews', 'qid_y': 'qid'})

In [20]:
df_clean_new

,article,pageviews,qid
0,Clara_Bow_(song),471741.0,Q124426354
15,Blackbird_(Beatles_song),64358.0,Q880719
45,Fortnight_(Taylor_Swift_song),56037.0,Q125511018
58,Karma_(JoJo_Siwa_song),38489.0,Q125450884
69,Texas_Hold_'Em_(song),36496.0,Q124507936
...,...,...,...
21047,In_the_Garden_(1912_song),90.0,Q11977017
21048,Bam_Bam_(Sister_Nancy_song),90.0,Q39069690
21049,Poison_(Alice_Cooper_song),90.0,Q508899
21050,Shelter_(Porter_Robinson_and_Madeon_song),90.0,Q27467421


In [21]:
df_clean_new.to_csv('articles_song.csv', index=True, header=True)

In [22]:
qid_list = [qid for qid in df_clean_new["qid"]]

In [23]:
qid_list

['Q124426354',
 'Q880719',
 'Q125511018',
 'Q125450884',
 'Q124507936',
 'Q125131153',
 'Q125503478',
 'Q466255',
 'Q969635',
 'Q125450884',
 'Q169298',
 'Q1573997',
 'Q1573997',
 'Q124422481',
 'Q3518110',
 'Q67184416',
 'Q122332978',
 'Q7959653',
 'Q124556312',
 'Q1813850',
 'Q700867',
 'Q1231722',
 'Q18208944',
 'Q155016',
 'Q125450571',
 'Q1165404',
 'Q125371893',
 'Q125024843',
 'Q22075121',
 'Q2001828',
 'Q115945493',
 'Q383842',
 'Q7516352',
 'Q125166898',
 'Q112577',
 'Q1142043',
 'Q114721508',
 'Q1932351',
 'Q312122',
 'Q20856833',
 'Q21172725',
 'Q123169436',
 'Q7289232',
 'Q19892111',
 'Q5679132',
 'Q1420677',
 'Q592696',
 'Q1764379',
 'Q744297',
 'Q3293037',
 'Q2430003',
 'Q3392431',
 'Q2311707',
 'Q2604122',
 'Q2137506',
 'Q1057533',
 'Q4929858',
 'Q1647391',
 'Q863433',
 'Q582848',
 'Q1121052',
 'Q1742084',
 'Q1513654',
 'Q964737',
 'Q999593',
 'Q641677',
 'Q1138836',
 'Q3059276',
 'Q104296021',
 'Q5976590',
 'Q3635151',
 'Q2078402',
 'Q1786535',
 'Q125347338',
 'Q1233120

The QID List needs to be cleaned again, there's a duplicated QID. 
Additionally, article needs to be cleaned in the query so that there's a ")" at the end of song

In [24]:
unique_list = []
for qid in qid_list:
    if qid not in unique_list:
        unique_list.append(qid)
    else:
        continue
unique_list

['Q124426354',
 'Q880719',
 'Q125511018',
 'Q125450884',
 'Q124507936',
 'Q125131153',
 'Q125503478',
 'Q466255',
 'Q969635',
 'Q169298',
 'Q1573997',
 'Q124422481',
 'Q3518110',
 'Q67184416',
 'Q122332978',
 'Q7959653',
 'Q124556312',
 'Q1813850',
 'Q700867',
 'Q1231722',
 'Q18208944',
 'Q155016',
 'Q125450571',
 'Q1165404',
 'Q125371893',
 'Q125024843',
 'Q22075121',
 'Q2001828',
 'Q115945493',
 'Q383842',
 'Q7516352',
 'Q125166898',
 'Q112577',
 'Q1142043',
 'Q114721508',
 'Q1932351',
 'Q312122',
 'Q20856833',
 'Q21172725',
 'Q123169436',
 'Q7289232',
 'Q19892111',
 'Q5679132',
 'Q1420677',
 'Q592696',
 'Q1764379',
 'Q744297',
 'Q3293037',
 'Q2430003',
 'Q3392431',
 'Q2311707',
 'Q2604122',
 'Q2137506',
 'Q1057533',
 'Q4929858',
 'Q1647391',
 'Q863433',
 'Q582848',
 'Q1121052',
 'Q1742084',
 'Q1513654',
 'Q964737',
 'Q999593',
 'Q641677',
 'Q1138836',
 'Q3059276',
 'Q104296021',
 'Q5976590',
 'Q3635151',
 'Q2078402',
 'Q1786535',
 'Q125347338',
 'Q123312089',
 'Q52808665',
 'Q104774

In [25]:
file_name = 'qid_list.txt'

with open(file_name, 'w') as file:
    for qid in unique_list:
        file.write(f"{qid}\n")


### Redoing and cleaning the data

In [14]:
df_2 = df_merged.drop(['qid_x'], axis=1)

In [15]:
df_clean2 = df_2.drop_duplicates()

In [16]:
df_clean2

,date,article,daily_pageviews,monthly_total,qid_y
0,2024-04-18,Clara_Bow_(song),2603,471741.0,Q124426354
1,2024-04-19,Clara_Bow_(song),3599,471741.0,Q124426354
2,2024-04-19,Clara_Bow_(song),238580,471741.0,Q124426354
3,2024-04-20,Clara_Bow_(song),105861,471741.0,Q124426354
4,2024-04-21,Clara_Bow_(song),59858,471741.0,Q124426354
...,...,...,...,...,...
21047,2024-04-14,When_I'm_Gone_(Eminem_song),90,90.0,Q2263502
21048,2024-04-19,Bam_Bam_(Sister_Nancy_song),90,90.0,Q39069690
21049,2024-04-22,Poison_(Alice_Cooper_song),90,90.0,Q508899
21050,2024-04-26,Shelter_(Porter_Robinson_and_Madeon_song),90,90.0,Q27467421


In [17]:
df_clean_new = df_clean2.rename(columns={'qid_y': 'qid'})

In [18]:
df_clean_new

,date,article,daily_pageviews,monthly_total,qid
0,2024-04-18,Clara_Bow_(song),2603,471741.0,Q124426354
1,2024-04-19,Clara_Bow_(song),3599,471741.0,Q124426354
2,2024-04-19,Clara_Bow_(song),238580,471741.0,Q124426354
3,2024-04-20,Clara_Bow_(song),105861,471741.0,Q124426354
4,2024-04-21,Clara_Bow_(song),59858,471741.0,Q124426354
...,...,...,...,...,...
21047,2024-04-14,When_I'm_Gone_(Eminem_song),90,90.0,Q2263502
21048,2024-04-19,Bam_Bam_(Sister_Nancy_song),90,90.0,Q39069690
21049,2024-04-22,Poison_(Alice_Cooper_song),90,90.0,Q508899
21050,2024-04-26,Shelter_(Porter_Robinson_and_Madeon_song),90,90.0,Q27467421


In [21]:
df_clean_new["article"] = (
    df_clean_new["article"]
    .astype(str)
    .str.encode('latin-1', errors='replace') # Treat the string characters as raw 'latin-1' bytes
    .str.decode('utf-8', errors='replace')   # Re-decode the resulting bytes as UTF-8
)

Helper function to clean the article titles, so it's just the songs

In [22]:
def clean_articles(article_title):
    clean_title = article_title.split("_(")[0]
    return clean_title

In [24]:
df_clean_new["article"] = df_clean_new["article"].apply(lambda x: clean_articles(x))

In [25]:
df_clean_new

,date,article,daily_pageviews,monthly_total,qid
0,2024-04-18,Clara_Bow,2603,471741.0,Q124426354
1,2024-04-19,Clara_Bow,3599,471741.0,Q124426354
2,2024-04-19,Clara_Bow,238580,471741.0,Q124426354
3,2024-04-20,Clara_Bow,105861,471741.0,Q124426354
4,2024-04-21,Clara_Bow,59858,471741.0,Q124426354
...,...,...,...,...,...
21047,2024-04-14,When_I'm_Gone,90,90.0,Q2263502
21048,2024-04-19,Bam_Bam,90,90.0,Q39069690
21049,2024-04-22,Poison,90,90.0,Q508899
21050,2024-04-26,Shelter,90,90.0,Q27467421


In [26]:
df_clean_new["article"] = df_clean_new["article"].str.replace("_", " ")

In [28]:
df_clean_new

,date,article,daily_pageviews,monthly_total,qid
0,2024-04-18,Clara Bow,2603,471741.0,Q124426354
1,2024-04-19,Clara Bow,3599,471741.0,Q124426354
2,2024-04-19,Clara Bow,238580,471741.0,Q124426354
3,2024-04-20,Clara Bow,105861,471741.0,Q124426354
4,2024-04-21,Clara Bow,59858,471741.0,Q124426354
...,...,...,...,...,...
21047,2024-04-14,When I'm Gone,90,90.0,Q2263502
21048,2024-04-19,Bam Bam,90,90.0,Q39069690
21049,2024-04-22,Poison,90,90.0,Q508899
21050,2024-04-26,Shelter,90,90.0,Q27467421


In [29]:
df_clean_new.to_csv('articles_song2.csv', index=True, header=True)

Running the entity collection just in case there are any changes

In [30]:
qid_list = [qid for qid in df_clean_new["qid"]]

In [31]:
unique_list = []
for qid in qid_list:
    if qid not in unique_list:
        unique_list.append(qid)
    else:
        continue
unique_list

['Q124426354',
 'Q880719',
 'Q125511018',
 'Q125450884',
 'Q124507936',
 'Q125131153',
 'Q125503478',
 'Q466255',
 'Q969635',
 'Q169298',
 'Q1573997',
 'Q124422481',
 'Q3518110',
 'Q67184416',
 'Q122332978',
 'Q7959653',
 'Q124556312',
 'Q1813850',
 'Q700867',
 'Q1231722',
 'Q18208944',
 'Q155016',
 'Q125450571',
 'Q1165404',
 'Q125371893',
 'Q125024843',
 'Q22075121',
 'Q2001828',
 'Q115945493',
 'Q383842',
 'Q7516352',
 'Q125166898',
 'Q112577',
 'Q1142043',
 'Q114721508',
 'Q1932351',
 'Q312122',
 'Q20856833',
 'Q21172725',
 'Q123169436',
 'Q7289232',
 'Q19892111',
 'Q5679132',
 'Q1420677',
 'Q592696',
 'Q1764379',
 'Q744297',
 'Q3293037',
 'Q2430003',
 'Q3392431',
 'Q2311707',
 'Q2604122',
 'Q2137506',
 'Q1057533',
 'Q4929858',
 'Q1647391',
 'Q863433',
 'Q582848',
 'Q1121052',
 'Q1742084',
 'Q1513654',
 'Q964737',
 'Q999593',
 'Q641677',
 'Q1138836',
 'Q3059276',
 'Q104296021',
 'Q5976590',
 'Q3635151',
 'Q2078402',
 'Q1786535',
 'Q125347338',
 'Q123312089',
 'Q52808665',
 'Q104774

In [32]:
file_name = 'qid_list2.txt'

with open(file_name, 'w') as file:
    for qid in unique_list:
        file.write(f"{qid}\n")

## Data Collection: Part 3 (Entire 2024 year)

In [6]:
query = """
SELECT
    -- 1. Extract the year to distinguish data across multiple years (if necessary)
    YEAR(t1.date) AS year,
    -- 2. Extract the month to define the aggregation period
    MONTH(t1.date) AS month,
    t1.article,
    -- 3. Retrieve the distinct QID (MIN or MAX works since QID is constant per article)
    MIN(t1.qid) AS qid,
    -- 4. Calculate the total pageviews for the group (year, month, article)
    SUM(t1.pageviews) AS monthly_pageviews
FROM
    data_table t1
WHERE
    -- Filter for articles that end specifically with '(song)'
    t1.article LIKE '%song)'
GROUP BY
    -- Aggregate all daily pageviews into monthly buckets for each article
    YEAR(t1.date),
    MONTH(t1.date),
    t1.article
ORDER BY
    year,
    month,
    monthly_pageviews DESC;
"""
result = conn.sql(query).df()
result

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,year,month,article,qid,monthly_pageviews
0,2024,1,The_Twelve_Days_of_Christmas_(song),Q112577,41500.0
1,2024,1,Leave_the_World_Behind_(song),Q23056,30525.0
2,2024,1,American_Pie_(song),Q466255,27442.0
3,2024,1,Hello_(Adele_song),Q21172725,27276.0
4,2024,1,Hallelujah_(Leonard_Cohen_song),Q969635,24706.0
...,...,...,...,...,...
20469,2024,12,Delilah_(Tom_Jones_song),Q903546,90.0
20470,2024,12,Miracles_(Jefferson_Starship_song),Q6872506,90.0
20471,2024,12,Best_of_My_Love_(The_Emotions_song),Q4896966,90.0
20472,2024,12,China_Grove_(song),Q5099707,90.0


In [14]:
result["article"] = (result["article"].astype(str).str
                     .encode('latin-1', errors='replace') # Treat the string characters as raw 'latin-1' bytes
                    .str.decode('utf-8', errors='replace')   # Re-decode the resulting bytes as UTF-8
)

In [15]:
def clean_articles(article_title):
    clean_title = article_title.split("_(")[0]
    return clean_title

In [16]:
result["article"] = result["article"].apply(lambda x: clean_articles(x))

In [17]:
result["article"] = result["article"].str.replace("_", " ")

In [18]:
result

,year,month,article,qid,monthly_pageviews
0,2024,1,The Twelve Days of Christmas,Q112577,41500.0
1,2024,1,Leave the World Behind,Q23056,30525.0
2,2024,1,American Pie,Q466255,27442.0
3,2024,1,Hello,Q21172725,27276.0
4,2024,1,Hallelujah,Q969635,24706.0
...,...,...,...,...,...
20469,2024,12,Delilah,Q903546,90.0
20470,2024,12,Miracles,Q6872506,90.0
20471,2024,12,Best of My Love,Q4896966,90.0
20472,2024,12,China Grove,Q5099707,90.0


In [ ]:
result.to_csv('articles_song3.csv', index=True, header=True)

In [7]:
qid_list = [qid for qid in result["qid"]]

In [9]:
len(qid_list)

20474

In [10]:
unique_list = []
for qid in qid_list:
    if qid not in unique_list:
        unique_list.append(qid)
    else:
        continue
unique_list

['Q112577',
 'Q23056',
 'Q466255',
 'Q21172725',
 'Q969635',
 'Q169298',
 'Q2991794',
 'Q3772414',
 'Q5291632',
 'Q1579259',
 'Q3518110',
 'Q744297',
 'Q67184416',
 'Q124363248',
 'Q3293037',
 'Q18208944',
 'Q123169436',
 'Q124369154',
 'Q700867',
 'Q7516352',
 'Q7959653',
 'Q1165404',
 'Q241339',
 'Q1971',
 'Q98066028',
 'Q123273186',
 'Q3503685',
 'Q592696',
 'Q1142043',
 'Q115945493',
 'Q124353181',
 'Q19892111',
 'Q123990023',
 'Q104296021',
 'Q2735688',
 'Q123990059',
 'Q1231722',
 'Q2604122',
 'Q863433',
 'Q1130943',
 'Q114721508',
 'Q2030651',
 'Q124550519',
 'Q312122',
 'Q124256479',
 'Q122332978',
 'Q121167968',
 'Q155016',
 'Q5183105',
 'Q1764379',
 'Q124385775',
 'Q124317039',
 'Q1513654',
 'Q23004224',
 'Q1057533',
 'Q122662730',
 'Q2078402',
 'Q383842',
 'Q5976590',
 'Q104774451',
 'Q4357239',
 'Q1786535',
 'Q1647391',
 'Q379751',
 'Q999593',
 'Q641677',
 'Q1138361',
 'Q2311707',
 'Q2639231',
 'Q2341319',
 'Q1742084',
 'Q1138836',
 'Q3392431',
 'Q578498',
 'Q1573997',
 'Q1

In [11]:
len(unique_list)

4462

In [12]:
file_name = 'qid_list3.txt'

with open(file_name, 'w') as file:
    for qid in unique_list:
        file.write(f"{qid}\n")